# copied from a previous rawnav repo - needs to be updated for the new workflow

In [1]:
import os, sys, glob, pandas as pd, geopandas as gpd
import pyarrow as pa, pyarrow.parquet as pq
import folium
from folium.plugins import MarkerCluster
from folium import CircleMarker
from IPython.display import IFrame

#### ben imports
import json
import numpy as np
from scipy.spatial import cKDTree
from shapely.geometry import LineString, Point


pd.set_option('display.max_columns', None)
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
%load_ext autoreload
%autoreload 2

#NAD83 Maryland (ft)
wmata_crs = 2248

In [2]:
from ipywidgets import interact
from shapely.ops import cascaded_union, linemerge

In [3]:
#os.path.pardir is equivalent to ".."
#need to append folder 2 levels up in order to load wmatarawnav
sys.path.append(os.path.join( os.path.pardir,os.path.pardir))
import wmatarawnav as wr

In [4]:
# 1.2 Set Global Parameters
############################################
if os.getlogin() == "WylieTimmerman":
    path_working = r"C:\OD\OneDrive - Foursquare ITP\Projects\WMATA_AVL"
    os.chdir(os.path.join(path_working))
    sys.path.append(r"C:\OD\OneDrive - Foursquare ITP\Projects\WMATA_AVL")
    path_sp = r"C:\Users\WylieTimmerman\Documents\projects_local\wmata_avl_local"
    path_source_data = os.path.join(path_sp,"data","00-raw")
    path_processed_data = os.path.join(path_sp, "data","02-processed")
    path_segments = os.path.join(path_working,"data","02-processed")
elif os.getlogin() == "abibeka":
    path_working = r"C:\Users\abibeka\OneDrive - Kittelson & Associates, Inc\Documents\Github\WMATA_AVL"
    os.chdir(os.path.join(path_working))
    sys.path.append(path_working)
    path_source_data = r"C:\Users\abibeka\OneDrive - Kittelson & Associates, Inc\Documents\WMATA-AVL\Data"
    path_processed_data = os.path.join(path_source_data, "ProcessedData")
    path_segments = path_processed_data
elif os.getlogin() == "E048374":
    # Working Paths
    path_working = r"C:\Users\E048374\OneDrive - WMATA\rawnav_rachel_fork\WMATA_AVL"
    os.chdir(os.path.join(path_working))
    sys.path.append(r"C:\Users\E048374\OneDrive - WMATA\rawnav_rachel_fork\WMATA_AVL")
    path_source_data = r"\\l-600730\RawNavArchive"
    path_sp = r"C:\Users\E048374\Documents\RawNav"
    path_processed_data = os.path.join(path_sp, "data", "02-processed")
    path_segments = path_processed_data
    
else:
    raise FileNotFoundError("Define the path_working, path_source_data, gtfs_dir, \
                            ZippedFilesloc, and path_processed_data in a new elif block")

## load segments for visualization

In [174]:
segments_loc = "C:\\Users\\E048374\\OneDrive - WMATA\\rawnav_rachel_fork\\WMATA_AVL\\data\\02-processed\\2020-09-08-segments_14th_only.geojson"
segments = gpd.read_file(segments_loc)

## load bus stops on south west side of 14th for southbound route

In [175]:
stop_seq_loc = "C:\\Users\\E048374\\OneDrive - WMATA\\rawnav_rachel_fork\\WMATA_AVL\\data\\02-processed\\route_52_stop_sequence.geojson"
stop_seq = gpd.read_file(stop_seq_loc)

In [176]:
stop_seq.dtypes

ROUTE_ID            object
ROUTE_VERSION        int64
DIRECTION           object
STOPID               int64
GEOID                int64
STOPORDER            int64
STOPSORTORDER        int64
LONGITUDE          float64
LATITUDE           float64
PATTERN_ID          object
GEODESCRIPTION      object
STOP_SEQUENCE      float64
REVENUE_FLAG         int64
geometry          geometry
dtype: object

In [177]:
#need to change datatype for merge with stop sequence below
stop_seq.loc[:,'PATTERN_ID'].unique()

array(['5201', '5202', '5203', '5204'], dtype=object)

In [178]:
stop_seq_subset = stop_seq[stop_seq.loc[:,'PATTERN_ID']=='5201'].copy()

In [179]:
def get_centroid_gpd(gpd_df):
    '''
    '''
    minx = gpd_df.bounds.mean()['minx']
    miny = gpd_df.bounds.mean()['miny']
    maxx = gpd_df.bounds.mean()['maxx']
    maxy = gpd_df.bounds.mean()['maxy']
    avg_x = np.array([minx, maxx]).mean()
    avg_y = np.array([miny, maxy]).mean()
    #return Lat, Long
    return[avg_y, avg_x]

In [180]:
esri_imagery = "https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}"
esri_attribution = \
    "Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, " \
    "UPR-EGP, and the GIS User Community"
this_map = folium.Map(location=get_centroid_gpd(stop_seq_subset),
                       tiles='cartodbdark_matter', zoom_start=16, max_zoom=25, control_scale=True)
folium.TileLayer(name="EsriImagery", tiles=esri_imagery, attr=esri_attribution,
                 zoom_start=16, max_zoom=25, control_scale=True).add_to(this_map)
folium.TileLayer('cartodbpositron', zoom_start=16, max_zoom=20, control_scale=True).add_to(this_map)
folium.TileLayer('openstreetmap', zoom_start=16, max_zoom=20, control_scale=True).add_to(this_map)

folium.Choropleth(
    segments,
    line_weight=3,
    line_color='blue'
).add_to(this_map)

h = folium.FeatureGroup(name='stop locations')

wr.plot_marker_clusters(this_map, stop_seq_subset, "LATITUDE","LONGITUDE",h, fill_color='#f79e19')

this_map.add_children(h)

# select closest stops to bus lane
there's a better, programmatic, way to do this but I just looked at the map, clicked on the close stops and I'm filtering for them below :)

In [181]:
geoid_list = [16203, 16662, 7352, 7243]
stop_seq_bus_lane = stop_seq_subset[stop_seq_subset.loc[:,'GEOID'].isin(geoid_list)].copy()

In [182]:
geoid_segment_dict = {16203:'14th_12', 16662:'14th_14', 7352:'14th_17', 7243:'14th_25'}

In [183]:
segment_geoid_dict = dict()
for keys, values in geoid_segment_dict.items():
    segment_geoid_dict[values] = keys

In [213]:
stop_seq_bus_lane.loc[:,'seg_name_id'] = stop_seq_bus_lane.loc[:,'GEOID'].apply(lambda x: geoid_segment_dict[x])
stop_seq_bus_lane.rename(columns={'ROUTE_ID':'route','ROUTE_VERSION':'pattern'
                                 ,'GEOID':'geoid','STOPID':'stop_id'},inplace=True)

In [216]:
segments_w_stops = segments.merge(stop_seq_bus_lane[['seg_name_id','geoid','stop_id','route','pattern']]
               , how='left'
              ,on='seg_name_id').copy()

In [217]:
segments_w_stops.loc[:,'geoid'] = segments_w_stops.loc[:,'geoid'].apply(lambda x: int(x) if x==x else 0).copy()
segments_w_stops.loc[:,'stop_id'] = segments_w_stops.loc[:,'stop_id'].apply(lambda x: int(x) if x==x else 0).copy()
segments_w_stops.loc[:,'route'] = '52'
segments_w_stops.loc[:,'pattern'] = 1
segments_w_stops

,seg_name_id,name_str,geometry,geoid,stop_id,route,pattern
0,14th_12,14th Street Northwest,"LINESTRING (-77.03254 38.92752, -77.03255 38.9...",16203,21627,52,1
1,14th_14,14th Street Northwest,"LINESTRING (-77.03254 38.92752, -77.03253 38.9...",16662,19066,52,1
2,14th_17,14th Street Northwest,"LINESTRING (-77.03234 38.92574, -77.03235 38.9...",7352,14083,52,1
3,14th_21,14th Street Northwest,"LINESTRING (-77.03234 38.92574, -77.03232 38.9...",0,0,52,1
4,14th_22,14th Street Northwest,"LINESTRING (-77.03232 38.92550, -77.03231 38.9...",0,0,52,1
5,14th_24,14th Street Northwest,"LINESTRING (-77.03224 38.92476, -77.03222 38.9...",0,0,52,1
6,14th_25,14th Street Northwest,"LINESTRING (-77.03222 38.92454, -77.03221 38.9...",7243,19143,52,1


In [239]:
segments_test = segments_w_stops[segments_w_stops.loc[:,'seg_name_id'].isin(['14th_22','14th_24','14th_25'])].copy()

In [240]:
segments_test.to_crs(2248, inplace=True)

In [241]:
segments_test.loc[:,'length'] = segments_test.loc[:,'geometry'].apply(lambda x: x.length)

In [242]:
segments_test

,seg_name_id,name_str,geometry,geoid,stop_id,route,pattern,length
4,14th_22,14th Street Northwest,"LINESTRING (1303138.895 458452.856, 1303140.67...",0,0,52,1,272.507588
5,14th_24,14th Street Northwest,"LINESTRING (1303159.851 458181.155, 1303166.08...",0,0,52,1,78.552727
6,14th_25,14th Street Northwest,"LINESTRING (1303166.082 458102.850, 1303168.09...",7243,19143,52,1,284.194639


In [243]:
segments_test.to_crs("EPSG:4326", inplace=True)
segments_test.reset_index(inplace=True, drop=True)

In [244]:
file_name = "seg_5201_by_intersection.geojson"
file_loc = f"C:\\Users\\E048374\\OneDrive - WMATA\\rawnav_rachel_fork\\WMATA_AVL\\data\\02-processed\\{file_name}"
segments_test.to_file(file_loc, driver='GeoJSON')

In [245]:
segments_test

,seg_name_id,name_str,geometry,geoid,stop_id,route,pattern,length
0,14th_22,14th Street Northwest,"LINESTRING (-77.03232 38.92550, -77.03231 38.9...",0,0,52,1,272.507588
1,14th_24,14th Street Northwest,"LINESTRING (-77.03224 38.92476, -77.03222 38.9...",0,0,52,1,78.552727
2,14th_25,14th Street Northwest,"LINESTRING (-77.03222 38.92454, -77.03221 38.9...",7243,19143,52,1,284.194639


In [246]:
stop_seq_pattern_seg = segments_test[['seg_name_id','geoid','stop_id','route','pattern']].copy()
stop_seq_pattern_seg

,seg_name_id,geoid,stop_id,route,pattern
0,14th_22,0,0,52,1
1,14th_24,0,0,52,1
2,14th_25,7243,19143,52,1


In [247]:
file_name = "stop_seq_pattern_5201_by_intersection.csv"
file_loc = f"C:\\Users\\E048374\\OneDrive - WMATA\\rawnav_rachel_fork\\WMATA_AVL\\data\\02-processed\\{file_name}"
stop_seq_pattern_seg.to_csv(file_loc, index=False)

In [248]:
consolidated_segments_df = segments_test[segments_test.loc[:,'seg_name_id']=='14th_25'][['seg_name_id'
                                                                                         ,'name_str'
                                                                                        ,'geoid', 'stop_id']].copy()

In [249]:
segments_test

,seg_name_id,name_str,geometry,geoid,stop_id,route,pattern,length
0,14th_22,14th Street Northwest,"LINESTRING (-77.03232 38.92550, -77.03231 38.9...",0,0,52,1,272.507588
1,14th_24,14th Street Northwest,"LINESTRING (-77.03224 38.92476, -77.03222 38.9...",0,0,52,1,78.552727
2,14th_25,14th Street Northwest,"LINESTRING (-77.03222 38.92454, -77.03221 38.9...",7243,19143,52,1,284.194639


In [250]:
consolidated_len = segments_test.loc[:,'length'].sum()
consolidated_geom = linemerge(cascaded_union(segments_test.geometry.values))

consolidated_seg_gdf = gpd.GeoDataFrame(consolidated_segments_df, crs="EPSG:4326", geometry=[consolidated_geom])
consolidated_seg_gdf.loc[:,'length'] = consolidated_len

In [252]:
consolidated_seg_gdf.reset_index(inplace=True, drop=True)

In [253]:
file_name = "seg_5201_by_stop.geojson"
file_loc = f"C:\\Users\\E048374\\OneDrive - WMATA\\rawnav_rachel_fork\\WMATA_AVL\\data\\02-processed\\{file_name}"
consolidated_seg_gdf.to_file(file_loc, driver='GeoJSON')

In [254]:
stop_seq_pattern = stop_seq_pattern_seg[stop_seq_pattern_seg.loc[:,'geoid']==7243].copy()

In [255]:
stop_seq_pattern.reset_index(inplace=True, drop=True)

In [256]:
file_name = "stop_seq_pattern_5201_by_stop.csv"
file_loc = f"C:\\Users\\E048374\\OneDrive - WMATA\\rawnav_rachel_fork\\WMATA_AVL\\data\\02-processed\\{file_name}"
stop_seq_pattern.to_csv(file_loc, index=False)